In [1]:
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics import accuracy_score
import os
import random
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [ ]:
samples_train = []
label_train = []
samples_test = []
label_test = []
img_trainsamples = []
dim = (100, 100)

i = 0
for filename in os.listdir(f'Dataset/'):
    cnt = 1
    for image in os.listdir(os.path.join(f'Dataset/',filename)):
        img = cv2.imread(os.path.join(f'Dataset/',filename,image))
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        gray_img = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY)
#         plt.imshow(gray_img, cmap='gray')
        gray_img_re = gray_img.reshape(-1)
        if cnt <= 80:
            img_trainsamples.append(gray_img)
            samples_train.append(gray_img_re)
            label_train.append(i)
        else:
            samples_test.append(gray_img_re)
            label_test.append(i)
        cnt += 1
    i = i + 1

x_train = np.array(samples_train) / 255
y_train = np.array(label_train).reshape(-1, 1)   

x_test = np.array(samples_test) / 255
y_test = np.array(label_test).reshape(-1, 1)

In [ ]:
T_train = np.zeros((max(y_train.shape), 19))
for i in range(19):
    T_train[y_train.reshape(max(y_train.shape)) == i, i] = 1

T_test = np.zeros((max(y_test.shape), 19))
for i in range(19):
    T_test[y_test.reshape(max(y_test.shape)) == i, i] = 1

In [ ]:
# Shuffle
randomize = np.arange(len(x_train))
np.random.shuffle(randomize)
x_train = x_train[randomize]
T_train = T_train[randomize]

In [ ]:
def binary_sigmoid(y_in):
    output = 1 / (1 + np.exp(-y_in))
    return output

In [2]:
def label_predict(y_out):
    Y = np.zeros((max(y_out.shape), 19))
    index = np.argmax(y_out, axis=1)
    for i in range(len(y_out)):
        Y[i, index[i]] = 1
    return Y

In [3]:
# weight = np.random.uniform(low=-0.000001, high=0.000001, size=(10000, 19))
weight = np.zeros((10000, 19))
# b = np.random.uniform( size=(19))
b = np.zeros((19))
alpha = 0.0001
len_pattern = len(x_train)

NameError: name 'x_train' is not defined

In [8]:
# alpha = 0.001

In [9]:
print(x_train.shape)
print(weight.shape)
print(x_test.shape)

(1520, 10000)
(10000, 19)
(380, 10000)


In [10]:
# largestchange = 100
Y_out = np.zeros((max(T_train.shape),19))
for epoch in range(300):
    for p in range(len_pattern):
        for j in range(19):
            y_in = b[j] + np.matmul(x_train[p, :], weight[:, j])
            y_out = binary_sigmoid(y_in)
            Y_out[p, j] = y_out
            # update weight
            delta_w = alpha * (T_train[p, j] - y_out)*y_out*(1-y_out)*x_train[p, :]
            w_new = weight[:, j] + delta_w
            weight[: , j] = w_new
            # update bias
            delta_b = alpha*(T_train[p, j] - y_out)*y_out*(1-y_out)
            b_new = b[j] + delta_b
            b[j] = b_new
    Y_pred = label_predict(Y_out)
    acc = accuracy_score(Y_pred, T_train)
    print("Epoch:", epoch+1, "Accuracy:", acc*100)

Epoch: 1 Accuracy: 6.513157894736842
Epoch: 2 Accuracy: 9.276315789473683


KeyboardInterrupt: 

In [24]:
# test data
y_out_test = np.zeros((max(T_test.shape),19))
len_x_test = len(x_test)
for p in range(len_x_test):
    for j in range(19):
        y_in = b[j] + np.matmul(x_test[p, :], weight[:, j])
        y_out_test[p, j] = binary_sigmoid(y_in)

Y_pred_test = label_predict(y_out_test)
acc_test = accuracy_score(Y_pred_test, T_test)
print("Accuracy on test data:", acc_test*100)

Accuracy on test data: 38.421052631578945


In [21]:
def add_noise(samples, ratio):
    row , col = samples[0].shape
    pixels = row * col
    num = int(pixels * ratio)
    noisy_samples = []
#     randomlist = random.sample(range(0, pixels), num)
    for i in range(len(samples)):
        img = samples[i]
        for j in range(num):
            # Pick a random y coordinate
            y_coord = random.randint(0, row - 1)

            # Pick a random x coordinate
            x_coord = random.randint(0, col - 1)

            if img[y_coord][x_coord] == 0:
                img[y_coord][x_coord] = 255

            elif img[y_coord][x_coord] == 255:
                img[y_coord][x_coord] = 0

            else:
                img[y_coord][x_coord] = random.choice([0, 255])

        noisy_samples.append(img)
    return noisy_samples

In [24]:
noisy_img = add_noise(img_trainsamples, 0.01)

In [26]:
noisy_samples = []

for i in range(len(noisy_img)):
    noisy_samples.append(noisy_img[i].reshape(-1))

x_noisy = np.array(noisy_samples) / 255

In [ ]:
# noisy data
y_out_noisy = np.zeros((max(T_train.shape),19))
len_x_noisy = len(x_noisy)
for p in range(len_x_noisy):
    for j in range(19):
        y_in = b[j] + x_noisy[p, :] @ weight[:, j]
        y_out_noisy[p, j] = binary_sigmoid(y_in)

Y_pred_noisy = label_predict(y_out_noisy)
acc_noisy = accuracy_score(Y_pred_noisy, T_train)
print("Accuracy on noisy data:", acc_noisy*100)